Triagram

In [20]:
#I am creating a triagram model on indian names
#I will also implement a thing that i thought he missed in the video and also will take into consideration the lenght of a name and not just take average negetive log likelyhood

In [21]:
from google.colab import drive
from pathlib import Path

drive.mount('/content/drive')

folder_path = Path("/content/drive/MyDrive/llm_week2/Triagram_week2_project1")
file_path = folder_path / "Indian_Names.txt"


lines = open(file_path, 'r').read().splitlines()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
words=[]
for i in lines:
  words.append(i.split(','))
words = [item for sublist in words for item in sublist]
words= words[1::2]
words.append('om')
for word in words:
  for char in word:
    if char == '0' or char == '8':
      words.remove(word)

In [23]:
import torch
import random
def matton(a):
  return a[0]*27 +a[1]
from numpy import shape

In [24]:
from math import exp
def  gaussi(x):
  #deviation,average,total
  pi = 3.1415
  ex = ((x-average)**2)/(2*var)
  mull = len(words)/(deviation*((2*pi)**0.5))
  return mull*exp(-ex)

In [25]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}

In [26]:

random.seed(200000)
random.shuffle(words)
train = words[:5177]
dev = words[5178:5825]
test = words[5826:]

In [27]:
import torch.nn.functional as F

# **After running cells till here you can go to the final cells and run them**

In [28]:
print(stoi)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}


In [29]:
N = torch.zeros((729, 27), dtype=torch.int32)
for w in train:
  chs = ['.']+['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    N[matton([ix1, ix2]), ix3] += 1


In [30]:

print(N[0])

tensor([  0, 474, 325, 109, 238,  14,  89, 193, 153,  81, 235, 303, 101, 436,
        295,  25, 304,   0, 426, 965, 145,  47, 135,  17,   0,  45,  22],
       dtype=torch.int32)


In [31]:
P = (N+1).float()
P /= P.sum(1, keepdims=True)

In [32]:
g = torch.Generator().manual_seed(2147483647)

for i in range(10):

  out = []
  ix = [0,0]
  while True:
    p = P[matton(ix)]
    i1 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(itos[i1])
    ix = [ix[1],i1]
    if i1 == 0 and ix[0]!=0:
      break
  print(''.join(out))
  # since we get very shit names lets try neural net

dexzmaloglkurajma.
karharvinimatta.
nalkfuk.
ka.
da.
samitaubjtbhrugotas.
mozimqctvujiwpteda.
ka.
hakamsideuenkavnyny.
fobspmhwciveevtariya.


In [33]:
log_likelihood = 0.0
n = 0

for w in train:
  chs = ['.']+['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    prob = P[matton([ix1,ix2]), ix3]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

for w in dev:
  chs = ['.']+['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    prob = P[matton([ix1,ix2]), ix3]
    logprob = torch.log(prob)
    log_likelihood += logprob
    n += 1
    #better than bigram even on a untrained dataset

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

log_likelihood=tensor(-82949.2266)
nll=tensor(82949.2266)
2.1778881549835205
log_likelihood=tensor(-93608.5547)
nll=tensor(93608.5547)
2.1850736141204834


**I am implementing the fact that the lenght of name matters and is a gaussian here**

In [34]:
sum=0
var= 0
for word in words:
  sum += len(word)
average =sum/len(words)
for word in words:
  var += (len(word)-average)**2
var = var/len(words)
deviation = var**0.5
print(average)
print(deviation)
print(var)
len(words)/(deviation*((2*3.1415)**0.5))

6.3533137648694575
1.6306091995849592
2.6588863617711014


1583.6973461427071

In [35]:
log_likelihood = 0.0
n = 0

for w in train:
  chs = ['.']+['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    prob = P[matton([ix1,ix2]), ix3]
    logprob = torch.log(prob).item()
    norm_logprob = logprob*(gaussi(average)/gaussi(len(w)))**0.1 # power here controls the weight of this factor before it was too high and i was only getting llll in the predictions
    # i am not really sure on how to use the gaussian i could do 1-gaussian or 1/ gaussian
    log_likelihood += norm_logprob
    n += 1

print(f'{log_likelihood=}')
nll = -log_likelihood
print(f'{nll=}')
print(f'{nll/n}')

log_likelihood=-88978.14417740515
nll=88978.14417740515
2.336181483902779


In [36]:
g = torch.Generator().manual_seed(2147483647)
W1 = torch.randn((729, 27), generator=g, requires_grad=True)

In [37]:
for k in range(10):
  loss1=0
  for w in train:
    xs1, ys1 = [], []
    chs = ['.']+['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
      ix1 = stoi[ch1]
      ix2 = stoi[ch2]
      ix3 = stoi[ch3]
      xs1.append(matton([ix1,ix2]))
      ys1.append(ix3)

    xs1 = torch.tensor(xs1)
    ys1 = torch.tensor(ys1)
    num = xs1.nelement()
    # forward pass
    xenc = F.one_hot(xs1, num_classes=729).float()
    logits = xenc @ W1
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdims=True)
    logprob = probs[torch.arange(num), ys1].log()
    norm_logprob = logprob*((gaussi(average)/gaussi(len(w)))**0.1)
    loss1 += -norm_logprob.mean() + 0.01*(W1**2).mean()
  loss1 = loss1/len(train)
  print(loss1.item())
  '''logprob = torch.log(prob).item()
  norm_logprob = logprob*(gaussi(1)/gaussi(len(w)))**0.4
  log_likelihood += norm_logprob'''
  # backward pass
  W1.grad = None
  loss1.backward()

  # update
  W1.data += -10 * W1.grad

4.0927324295043945
4.042555809020996
3.9986701011657715
3.9594266414642334
3.9238216876983643
3.8912203311920166
3.861217498779297
3.8335349559783936
3.807934284210205
3.7842416763305664


In [38]:
g = torch.Generator().manual_seed(2147483647)

for i in range(5):

  out = []
  ix = [0,0]
  while True:
    xenc = F.one_hot(torch.tensor([matton(ix)]), num_classes=729).float()
    logits = xenc @ W1
    counts = logits.exp()
    p = counts / counts.sum(1, keepdims=True)
    # ----------

    i1 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    ix= [ix[1],i1]
    out.append(itos[i1])
    if i1 == 0 and ix[0]!=0:
      break
  print(''.join(out))

texzdfzjglkurxycczkwyhhmvlzimjtnagnrlkfdkzka.
da.
slaxvoubjtbhrggitmj.
idzjedktvojkw.
t.


In [39]:
xs, ys = [], []

for w in train:
  chs = ['.']+['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs.append(matton([ix1,ix2]))
    ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((729, 27), generator=g, requires_grad=True)

number of examples:  38087


In [40]:
for k in range(5):

  # forward pass
  xenc = F.one_hot(xs, num_classes=729).float()
  logits = xenc @ W
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  print(loss.item())
  '''logprob = torch.log(prob).item()
  norm_logprob = logprob*(gaussi(1)/gaussi(len(w)))**0.4
  log_likelihood += norm_logprob'''
  # backward pass
  W.grad = None
  loss.backward()

  # update
  W.data += -10 * W.grad

3.864635944366455
3.821072816848755
3.7823805809020996
3.7473745346069336
3.7153143882751465


In [41]:
g = torch.Generator().manual_seed(2147483647)

for i in range(20):

  out = []
  ix = [0,0]
  while True:
    xenc = F.one_hot(torch.tensor([matton(ix)]), num_classes=729).float()
    logits = xenc @ W
    counts = logits.exp()
    p = counts / counts.sum(1, keepdims=True)
    # ----------

    i1 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    ix= [ix[1],i1]
    out.append(itos[i1])
    if i1 == 0 and ix[0]!=0:
      break
  print(''.join(out))

texzdfzjglkurxycczkwyhhmvlzimjtnagnrlkfdkzka.
za.
royxcpbbpwkhrggitmj.
idzjedktvojkw.
t.
dagkkjeyktgscdgu.
nkgvnrnfrqtbspmhwcjdewvtahlvsuqysfxxblgjxlhgfiwuidwnnjgpfdnipkezktsdesu.
zixht.
gbckgjbhzabsvoth.
kuybtqevacrftxuqyfhfsqenvxmvpfoqzmtrfvjbsdblmyvoxngsptjapxzqbgpqlhaifyemtlsiiimvlbdmlbyaduaqakmacgamcxwmcutssycxfmbalcsmhw.
jmyzqizfz.
bqzszelndchyy.
pcjxjuowyvtmmioatdohznicgubgyycfoovaccaj.
npuupstpicqdqdkb.
rdubmknomf.
pxfacdptadzjmlkdj.
gv.
zryilmugrlrkkmpzugrodpvxh.
ydevc.
gixgoseqr.


In [42]:
xs, ys = [], []

for w in dev:
  chs = ['.']+['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs.append(matton([ix1,ix2]))
    ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

xenc = F.one_hot(xs, num_classes=729).float()
logits = xenc @ W
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
loss = -probs[torch.arange(num), ys].log().mean()
print(loss.item())

number of examples:  4753
3.668745279312134


In [43]:
xs, ys = [], []

for w in dev:
  chs = ['.']+['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs.append(matton([ix1,ix2]))
    ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
print('number of examples: ', num)

xenc = F.one_hot(xs, num_classes=729).float()
logits = xenc @ W1
counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
loss = -probs[torch.arange(num), ys].log().mean()
print(loss.item())
#traditional loss is also better

number of examples:  4753
3.540224075317383


In [44]:
#loss incorperating gaussian
loss1=0
for w in dev:
  xs1, ys1 = [], []
  chs = ['.']+['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs1.append(matton([ix1,ix2]))
    ys1.append(ix3)

  xs1 = torch.tensor(xs1)
  ys1 = torch.tensor(ys1)
  num = xs1.nelement()
  # forward pass
  xenc = F.one_hot(xs1, num_classes=729).float()
  logits = xenc @ W1
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  logprob = probs[torch.arange(num), ys1].log()
  norm_logprob = logprob*((gaussi(average)/gaussi(len(w)))**0.1)
  loss1 += -norm_logprob.mean()
print(loss1.item())

2415.176025390625


# F.one_hot can be replaced by

In [45]:
loss1=0
for w in dev:
  xs1, ys1 = [], []
  chs = ['.']+['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs1.append(matton([ix1,ix2]))
    ys1.append(ix3)

  #xs1 = torch.tensor(xs1)
  ys1 = torch.tensor(ys1)
  num = ys1.nelement()

  logits = W1[xs1]#without one hot

  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  logprob = probs[torch.arange(num), ys1].log()
  norm_logprob = logprob*((gaussi(average)/gaussi(len(w)))**0.1)
  loss1 += -norm_logprob.mean()
loss1 = loss1/len(dev)
print(loss1.item())

3.732884168624878


In [46]:
xs, ys = [], []

for w in dev:
  chs = ['.']+['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs.append(matton([ix1,ix2]))
    ys.append(ix3)

#xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = ys.nelement()
print('number of examples: ', num)

logits = W1[xs]
loss = F.cross_entropy(logits,ys)
print(loss.item())

number of examples:  4753
3.5402238368988037


# **Run this for final result**

In [47]:
weights_with_gauss = folder_path / "Weights1.pt"
weights_without_gauss= folder_path / "Weights_without_gaussi.pt"
probabilities= folder_path / "probs.pt"
T= torch.load(weights_with_gauss) # load any of the saved weights

In [48]:
xs, ys = [], []

for w in test:
  chs = ['.']+['.'] + list(w) + ['.']
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = stoi[ch1]
    ix2 = stoi[ch2]
    ix3 = stoi[ch3]
    xs.append(matton([ix1,ix2]))
    ys.append(ix3)

ys = torch.tensor(ys)
num = ys.nelement()
print('number of examples: ', num)

logits = T[xs]
loss = F.cross_entropy(logits,ys)
print(loss.item())

number of examples:  4744
2.375758647918701
